In [4]:
#tu rzeczy do instalacji, zaczynac od !
%pip uninstall tensorflow
#%pip install matplotlib
#%pip install pandas
#tu sklearn, możliwe że trzeba zrestartowac krenel po updacie
#%pip install -U scikit-learn

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
#tu biblioteki
import numpy as np
import pandas as pd
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print("Import done")

ModuleNotFoundError: No module named 'tensorflow.python._pywrap_tfe'

In [ ]:
#GPUs - tu poki co nie dziala, trzeba postawic srodowisko wirtualne
gpus = tf.config.experimental.list_physical_devices('GPU')
print (len(gpus), " GPUs found!")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), " Physical GPUs, ", len(logical_gpus), " Logical GPUs")
    except RuntimeError as e:
        print(e)
print("GPUs done")

In [ ]:
def directory_flow(train_directory, test_directory, val_directory, shape):
    #directory = nazwy folderow,
    #shape = rozmiar obrazka, u nas 256x256
    print ("generators start")
    image_generator=ImageDataGenerator()
    train_gen = image_generator.flow_from_directory(train_directory, target_size=shape, shuffle=True) #sciezka do train dir
    print ("train done")
    val_gen = image_generator.flow_from_directory(val_directory, target_size=shape, shuffle=True)
    print ("validate done")
    test_gen = image_generator.flow_from_directory(test_directory, target_size=shape, shuffle=True) #sciezka do test dir
    print ("generators done")
    #zastanowic sie nad zwracaniem validate gen
    return train_gen, test_gen, val_gen

In [ ]:
#nie zadziala, bo jeszcze nie ma obrazkow
train_gen, test_gen, val_gen = directory_flow("./archive/TrainingData", "./archive/TestingData", "./archive/ValidationData", (224, 224))
print("Directory flow done!")

In [ ]:
#model
def make_model(shape):
    #te rzeczy w nawiasie zapobiegna przetrenowaniu
    base_model = tf.keras.applications.xception.Xception(include_top=False, weights="imagenet",input_shape=shape, pooling='max')
    model = Sequential([
        base_model,
        BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
        Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
        Dropout(rate= 0.45, seed= 123),
        Dense(15, activation= 'softmax')
        ])
    #ograniczenie warstw zapobiegnie przetrenowaniu
    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")

In [ ]:
model=make_model((224, 224, 3))
print("Model generated")

In [ ]:
model.summary()

In [ ]:
#zadziala jak wrzuce obrazki
model.fit(x=train_gen, epochs=40, validation_data=(val_gen),validation_steps= None, shuffle= False)